In [2]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import pyarrow
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
file_name = "allo_cine"
version="10"

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get(file_name, version=version)
print(data_asset.path)


df = pd.read_parquet(data_asset.path)

df.head()


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


azureml://subscriptions/72eb7803-e874-44cb-b6d9-33f2fa3eb88c/resourcegroups/vpoutotrg/workspaces/mlstudio-groupe4/datastores/workspaceblobstore/paths/UI/2025-04-08_094406_UTC/allocine_spider.parquet


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,director,editor,french_boxoffice,french_first_week_boxoffice,french_visa,genre,langage,length,nationality,title,url,viewers_score,vo_title
0,George Clooney|Julia Roberts|Jack O'Connell,3.5,2016-05-12,Jodie Foster,Sony Pictures Releasing France,847743,305385.0,144018,Thriller,Anglais,99,U.S.A.,Money Monster,/video/player_gen_cmedia=19561526&cfilm=214139...,3.6,None
1,Kad Merad|Géraldine Pailhas|Lola Creton,2.7,2015-01-21,Christophe Lamotte,Rezo Films,44310,29265.0,117439,Drame|Thriller,Français,100,France,Disparue en hiver,/article/fichearticle_gen_carticle=18639324.html,2.3,None
2,Nicolas Cage|Kev Adams|Ryan Reynolds,3.8,2013-04-10,Chris Sanders,Twentieth Century Fox France,2350349,501465.0,135882,Aventure|Animation|Comédie|Famille,Anglais,98,U.S.A.,Les Croods,/video/player_gen_cmedia=19410286&cfilm=146916...,3.9,The Croods
3,Dylan O'Brien|Michael Keaton|Taylor Kitsch,2.3,2017-09-20,Michael Cuesta,Metropolitan FilmExport,260465,104402.0,147308,Action|Thriller,Anglais,112,U.S.A.,American Assassin,/video/player_gen_cmedia=19569927&cfilm=194970...,3.2,None
4,Virginie Efira|Anaïs Demoustier|Laurent Stocker,3.5,2015-04-22,Emmanuel Mouret,Pyramide Distribution,125641,64046.0,138835,Comédie|Romance,Français,100,France,Caprice,/diaporamas/cinema/diaporama-18643599/,2.7,None


In [4]:
df['viewers_score']= pd.to_numeric(df['viewers_score'], errors='coerce')
df['critics_score']= pd.to_numeric(df['critics_score'], errors='coerce')
df['french_boxoffice']= pd.to_numeric(df['french_boxoffice'], errors='coerce')

In [7]:
df['length']= pd.to_numeric(df['length'], errors='coerce')
df['length'] = df['length'].apply(lambda x : np.nan if x == 0 else x )

In [8]:

df['date']= pd.to_datetime(df['date'], errors='coerce')

In [9]:
df['genre'] = df['genre'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['nationality'] = df['nationality'].str.split('|')
df['langage'] = df['langage'].str.split('|') 	

In [10]:
df = df[df['french_first_week_boxoffice'].notna()]
df = df[df['date'].notna()]


In [11]:
feature_of_interest = [
    'actors',
    'critics_score',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']


target = 'french_first_week_boxoffice'
data = feature_of_interest
data.append(target)

data.extend(info_film)
df_to_train = df[data]

In [12]:
df_to_train

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[George Clooney, Julia Roberts, Jack O'Connell]",3.5,2016-05-12,Jodie Foster,Sony Pictures Releasing France,[Thriller],[Anglais],99.0,[U.S.A.],3.6,305385.0,144018,Money Monster,None,/video/player_gen_cmedia=19561526&cfilm=214139...
1,"[Kad Merad, Géraldine Pailhas, Lola Creton]",2.7,2015-01-21,Christophe Lamotte,Rezo Films,"[Drame, Thriller]",[Français],100.0,[France],2.3,29265.0,117439,Disparue en hiver,None,/article/fichearticle_gen_carticle=18639324.html
2,"[Nicolas Cage, Kev Adams, Ryan Reynolds]",3.8,2013-04-10,Chris Sanders,Twentieth Century Fox France,"[Aventure, Animation, Comédie, Famille]",[Anglais],98.0,[U.S.A.],3.9,501465.0,135882,Les Croods,The Croods,/video/player_gen_cmedia=19410286&cfilm=146916...
3,"[Dylan O'Brien, Michael Keaton, Taylor Kitsch]",2.3,2017-09-20,Michael Cuesta,Metropolitan FilmExport,"[Action, Thriller]",[Anglais],112.0,[U.S.A.],3.2,104402.0,147308,American Assassin,None,/video/player_gen_cmedia=19569927&cfilm=194970...
4,"[Virginie Efira, Anaïs Demoustier, Laurent Sto...",3.5,2015-04-22,Emmanuel Mouret,Pyramide Distribution,"[Comédie, Romance]",[Français],100.0,[France],2.7,64046.0,138835,Caprice,None,/diaporamas/cinema/diaporama-18643599/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7422,"[Charlotte Gainsbourg, Stellan Skarsgård, Stac...",3.2,2014-01-01,Lars von Trier,Les Films du Losange,"[Drame, Erotique]",[Anglais],117.0,"[Danemark, Allemagne, France, Belgique, Grande...",3.3,60511.0,134563,Nymphomaniac - Volume 1,Nymphomaniac : Volume I,/article/fichearticle_gen_carticle=1000104999....
7423,"[Zabou Breitman, Adèle Exarchopoulos, Tchéky K...",2.3,2013-02-13,Nolwenn Lemesle,L.C.J. Editions et Productions,"[Comédie, Drame]",[Français],90.0,[France],2.6,3745.0,125505,Des Morceaux de Moi,None,/article/fichearticle_gen_carticle=18627659.html
7424,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",3.4,2012-03-21,Gary Ross,Metropolitan FilmExport,"[Action, Drame, Science Fiction]",[Anglais],142.0,[U.S.A.],3.8,509426.0,131961,Hunger Games,The Hunger Games,/video/player_gen_cmedia=19253059&cfilm=145083...
7425,"[Leonardo DiCaprio, Jonah Hill, Margot Robbie]",4.0,2013-12-25,Martin Scorsese,Metropolitan FilmExport,"[Biopic, Drame]",[Anglais],179.0,[U.S.A.],4.3,1013324.0,138743,Le Loup de Wall Street,The Wolf Of Wall Street,/video/player_gen_cmedia=19540275&cfilm=127524...


In [14]:
table = pa.Table.from_pandas(df_to_train)
pq.write_table(table, 'allocine_spider_clean.parquet')

In [4]:
df2 = pd.read_csv("ecran_total_spider.csv")
df2.head(5)


,date,genre,title,url_picture
0,16/04/2025,"Biopic, Drame",OXANA,https://ecran-total.fr/wp-content/uploads/2023...
1,16/04/2025,Comédie dramatique,LA RÉPARATION,https://ecran-total.fr/wp-content/uploads/2024...
2,16/04/2025,"Drame, Western",HARVEST,https://ecran-total.fr/wp-content/uploads/2024...
3,16/04/2025,Comédie,LE MÉLANGE DES GENRES,https://ecran-total.fr/wp-content/uploads/2024...
4,16/04/2025,"Drame, Horreur, Thriller",SINNERS,https://ecran-total.fr/wp-content/uploads/2024...


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         276 non-null    object
 1   genre        276 non-null    object
 2   title        276 non-null    object
 3   url_picture  276 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB


In [10]:
df2['date']=pd.to_datetime(df2['date'], errors = 'coerce')
df2['year']=df2['date'].dt.year

/tmp/ipykernel_75859/711646458.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df2['date']=pd.to_datetime(df2['date'], errors = 'coerce')


In [12]:
df2.head(20)

,date,genre,title,url_picture,year
0,2025-04-16,"Biopic, Drame",OXANA,https://ecran-total.fr/wp-content/uploads/2023...,2025
1,2025-04-16,Comédie dramatique,LA RÉPARATION,https://ecran-total.fr/wp-content/uploads/2024...,2025
2,2025-04-16,"Drame, Western",HARVEST,https://ecran-total.fr/wp-content/uploads/2024...,2025
3,2025-04-16,Comédie,LE MÉLANGE DES GENRES,https://ecran-total.fr/wp-content/uploads/2024...,2025
4,2025-04-16,"Drame, Horreur, Thriller",SINNERS,https://ecran-total.fr/wp-content/uploads/2024...,2025
5,2025-04-16,Non classé,KYUKA – AVANT LA FIN DE L’ÉTÉ,https://ecran-total.fr/wp-content/uploads/2024...,2025
6,2025-04-16,Action,RAPIDE,https://ecran-total.fr/wp-content/uploads/2024...,2025
7,2025-04-16,Comédie,AIMONS-NOUS VIVANTS,https://ecran-total.fr/wp-content/uploads/2024...,2025
8,2025-04-16,Drame,TOXIC,https://ecran-total.fr/wp-content/uploads/2024...,2025
9,2025-04-16,Drame,DIMANCHES,https://ecran-total.fr/wp-content/uploads/2024...,2025
